In [1]:
import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.model import BlueBrainCircuitModel
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light\
    import circuit_composition, circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')


A few tests for `class CellType`.

In [2]:
from neuro_dmt.models.bluebrain.circuit.model.cell_type import CellType
from collections import OrderedDict

cell_types = (
    CellType(OrderedDict((("mtype", "L23_MC"), ("layer", "L3"),     ("etype", "bNAC")))),
    CellType(OrderedDict((("mtype", "L23_MC"), ("etype", "bNAC"),   ("layer", "L3")))),
    CellType(OrderedDict((("layer", "L3"),     ("mtype", "L23_MC"), ("etype", "bNAC")))),
    CellType(OrderedDict((("layer", "L3"),     ("etype", "bNAC"),   ("mtype", "L23_MC")))),
    CellType(OrderedDict((("etype", "bNAC"),   ("mtype", "L23_MC"), ("layer", "L3")))),
    CellType(OrderedDict((("etype", "bNAC"),   ("layer", "L3"),     ("mtype", "L23_MC")))))

for cell_type in cell_types:
    assert isinstance(cell_type.value, pd.Series)
    assert len(cell_type.value) == 3
    
assert cell_types[0].specifier == ("mtype", "layer", "etype"), cell_types[1].specifier 
assert cell_types[1].specifier == ("mtype", "etype", "layer"), cell_types[2].specifier 
assert cell_types[2].specifier == ("layer", "mtype", "etype"), cell_types[3].specifier 
assert cell_types[3].specifier == ("layer", "etype", "mtype"), cell_types[4].specifier 
assert cell_types[4].specifier == ("etype", "mtype", "layer"), cell_types[5].specifier 
assert cell_types[5].specifier == ("etype", "layer", "mtype"), cell_types[6].specifier 


cell_types = (
    CellType(pd.Series(["L23_MC", "L3", "bNAC"], index=["mtype", "layer", "etype"])),
    CellType(pd.Series(["L23_MC", "bNAC", "L3"], index=["mtype", "etype", "layer"])),
    CellType(pd.Series(["L3", "L23_MC", "bNAC"], index=["layer", "mtype", "etype"])),
    CellType(pd.Series(["L3", "bNAC", "L23_MC"], index=["layer", "etype", "mtype"])),
    CellType(pd.Series(["bNAC", "L23_MC", "L3"], index=["etype", "mtype", "layer"])),
    CellType(pd.Series(["bNAC", "L3", "L23_MC"], index=["etype", "layer", "mtype"])))

for cell_type in cell_types:
    assert isinstance(cell_type.value, pd.Series)
    assert len(cell_type.value) == 3
    
assert cell_types[0].specifier == ("mtype", "layer", "etype"), cell_types[1].specifier 
assert cell_types[1].specifier == ("mtype", "etype", "layer"), cell_types[2].specifier 
assert cell_types[2].specifier == ("layer", "mtype", "etype"), cell_types[3].specifier 
assert cell_types[3].specifier == ("layer", "etype", "mtype"), cell_types[4].specifier 
assert cell_types[4].specifier == ("etype", "mtype", "layer"), cell_types[5].specifier 
assert cell_types[5].specifier == ("etype", "layer", "mtype"), cell_types[6].specifier 



We can define a `BlueBrainCircuitModel` instance for `MockCircuit`:

In [3]:
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
circuit_model =\
    BlueBrainCircuitModel(
        mock_circuit,
        label="BlueBrainCircuitModelMock")

In [4]:
cells = circuit_model.cells
conn_prob = circuit_model.connection_probability
for pairs in circuit_model.connection_probability.get(
    pre_synaptic_cells=cells.sample(n=100).rename(columns=conn_prob._at("pre_synaptic")),
    post_synaptic_cells=cells.sample(n=100).rename(columns=conn_prob._at("post_synaptic"))):
    pass
pre_synaptic_cells = cells.sample(n=100).rename(columns=conn_prob._at("pre_synaptic"))
post_synaptic_cells = cells.sample(n=100).rename(columns=conn_prob._at("post_synaptic"))
pd.concat(
    [pairs[conn_prob.measurement_variables].agg(conn_prob.aggregators).transpose()
     for pairs in conn_prob.get(pre_synaptic_cells, post_synaptic_cells)],
).reset_index(
    drop=True
).rename(
    columns=conn_prob.columns
).agg("sum")

total        10000
connected      202
dtype: int64

In [5]:
conn_prob(pre_synaptic_cells, post_synaptic_cells)

pre_synaptic_cell_type_specifier ()
post_synaptic_cell_type_specifier ()


total        10000
connected      202
dtype: int64

In [6]:
result_series = conn_prob(
    pd.Series({"mtype": "L23_MC"}),
    pd.Series({"mtype": "L23_MC"}), 
    number=100)

pre_synaptic_cell_type_specifier ('mtype',)
post_synaptic_cell_type_specifier ('mtype',)


In [7]:
print(conn_prob.circuit_model.cell_collection.get({"mtype": "L23_MC"}).mtype.unique())
print(conn_prob.circuit_model.get_cells(**{"mtype": "L23_MC"}).mtype.unique())

['L23_MC']
['L23_MC']


In [8]:
result_dataframes = conn_prob(
    cells,
    cells,
    groupby=conn_prob.GroupByVariables(("mtype", ), ("mtype", )),
    number=100,
    resample=True)

pre_synaptic_cell_type_specifier ('mtype',)
post_synaptic_cell_type_specifier ('mtype',)


In [9]:
result_dataframes.sum()

pairs                   total        10000.000000
                        connected      169.000000
connection_probability                   4.533511
dtype: float64

In [10]:
result_no_cell_groups = conn_prob(
    groupby=conn_prob.GroupByVariables(("mtype", ), ("mtype", )),
    resample=True,
    number=100)

pre_synaptic_cell_type_specifier ('mtype',)
post_synaptic_cell_type_specifier ('mtype',)


In [11]:
result_no_cell_groups.head()

pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L1_DAC             L23_ChC                 4       0.0                    0.0
                   L23_MC                  4       0.0                    0.0
                   L23_SBC                 4       0.0                    0.0
                   L4_LBC                  2       0.0                    0.0
                   L4_NGC                  4       0.0                    0.0

In [ ]:
result_no_cell_groups_no_types = conn_prob(number=1000)

pre_synaptic_cell_type_specifier ()
post_synaptic_cell_type_specifier ()


In [18]:
result_no_cell_groups_no_types

total        10000
connected      205
dtype: int64

## Pathway property

A `PathwayProperty` instance is a callable -- and callable:
```
class PathwayProperty(WithFields):
    """
    Compute and store a circuit's pathway properties.
    """
    phenomenon = Field(
        """
        A label to go with the pathway property.
        """)
    circuit_model = Field(
        """
        The circuit model for which this `PathwayProperty` has been defined.
        """)
    max_length_cell_type_specification = Field(
        """
        Max length of the cell type specifier that can be stored.
        """,
        __default_value__=2)

    def __call__(self,
            pre_synaptic_cell_group,
            post_synaptic_cell_group,
            by=None,
            given=None,
            resample=False,
            sampling_methodology=SamplingMethodology.random,
            number=100):
        """
        Compute, or retrieve from the store...

        Arguments
        -------------
        pre_synaptic_cell_group:  Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        post_synaptic_cell_group: Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        by : A property of a connection ((pre_synaptic_cell, post_synaptic_cell).
             Pathway property will be computed for given values of this variable.
             If a list of variables, the computation will be conditioned on all
             the variables.
        given: A dict providing variables and their values for which the
               the pathway property will be returned.
               
        Returns
        ---------------
        pd.Series if 
        
        """
```



In [18]:
s = circuit_model.connection_probability(
    np.array([1,2]),
    np.array([3,4]))

In [19]:
s

(number_pairs, total)        4.0
(number_pairs, connected)    0.0
connection_probability       0.0
dtype: float64

In [6]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import SamplingMethodology
pre_cell_group = pd.Series({"mtype": "L23_MC"})
post_cell_group = pd.Series({"mtype": "L23_MC"})
s = circuit_model.connection_probability(
    pre_cell_group,
    post_cell_group,
    sampling_methodology=SamplingMethodology.exhaustive)

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/model/pathway.py:354: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  pre_synaptic_cell_type_specifier


In [7]:
mock_circuit.cells.get({"mtype": "L23_ChC"}).head()

,region,layer,nucleus,mtype,etype,morph_class,synapse_class,x,y,z
227,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,123.384590,1651.865006,78.064000
228,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,56.219132,1692.110101,143.581369
229,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,191.856599,1623.164846,165.929919
230,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,135.442984,1626.620538,236.819712
231,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,174.672321,1606.008151,175.396775


In [8]:
mock_circuit.cells.get(mock_circuit.connectome.afferent_gids(140)).head()

,region,layer,nucleus,mtype,etype,morph_class,synapse_class,x,y,z
56,brain,1,not-defined,L1_DAC,not-defined,not-defined,INH,152.331923,1744.215587,154.261640
65,brain,1,not-defined,L1_DAC,not-defined,not-defined,INH,99.036326,1816.659774,189.228807
71,brain,1,not-defined,L1_SLAC,not-defined,not-defined,INH,179.609974,1818.600021,105.986680
257,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,206.052445,1643.353590,29.976529
297,brain,2,not-defined,L23_ChC,not-defined,not-defined,INH,226.179319,1631.551780,236.097489


In [9]:
s.xs("L1_DAC", level="post_synaptic_mtype")

number_pairs           connection_probability
                          total connected                       
pre_synaptic_mtype                                              
L1_DAC                     4900      98.0               0.020000
L1_SLAC                    4900      96.0               0.019592
L23_ChC                   11480     202.0               0.017596
L23_MC                    11480     245.0               0.021341
L23_SBC                   11480     253.0               0.022038
L4_LBC                     6650     124.0               0.018647
L4_NGC                     6650     125.0               0.018797
L4_TPC                   127890    2535.0               0.019822
L5_DBC                    17290     327.0               0.018913
L5_MC                     17290     314.0               0.018161
L5_TPC:A                  56770    1095.0               0.019288
L5_TPC:B                  56770    1131.0               0.019922
L5_UPC                    56770    1077.0               0.018971
L6_ChC                    15680     302.0               0.019260
L6_IPC                    74620    1493.0               0.020008
L6_MC                     15680     300.0               0.019133
L6_TPC:A                  74620    1405.0               0.018829
L6_TPC:B                  74620    1414.0               0.018949
L6_UPC                    74620    1418.0               0.019003

In [10]:
l1dacs = mock_circuit.cells.get({"mtype": "L1_DAC"}).index.values

In [11]:
l1dac_affs = np.concatenate(
    [mock_circuit.connectome.afferent_gids(post_gid)
     for post_gid in l1dacs],
    axis=0)

In [12]:
l1dac_aff_mtypes = mock_circuit.cells.get(l1dac_affs, ["mtype"])

In [13]:
l1dac_aff_mtypes.mtype.value_counts()

L4_TPC      2535
L6_IPC      1493
L6_UPC      1418
L6_TPC:B    1414
L6_TPC:A    1405
L5_TPC:B    1131
L5_TPC:A    1095
L5_UPC      1077
L5_DBC       327
L5_MC        314
L6_ChC       302
L6_MC        300
L23_SBC      253
L23_MC       245
L23_ChC      202
L4_NGC       125
L4_LBC       124
L1_DAC        98
L1_SLAC       96
Name: mtype, dtype: int64

In [14]:
mock_circuit.cells.get().mtype.value_counts()

L4_TPC      1827
L6_UPC      1066
L6_TPC:B    1066
L6_IPC      1066
L6_TPC:A    1066
L5_TPC:A     811
L5_UPC       811
L5_TPC:B     811
L5_DBC       247
L5_MC        247
L6_ChC       224
L6_MC        224
L23_MC       164
L23_ChC      164
L23_SBC      164
L4_NGC        95
L4_LBC        95
L1_SLAC       70
L1_DAC        70
Name: mtype, dtype: int64

In [15]:
127890 / 70

1827.0

In [16]:
circuit_model.get_connection_probability(
    pre_cell_group, post_cell_group)

NameError: name 'post_cells' is not defined

In [ ]:
def get_connection_probability(
            pre_cell_type_specifier,
            post_cell_type_specifier):
        """
        Compute connection probability once for either the entire
        circuit, or samples of afferent, or samples of  efferent,
        or samples of both efferent and afferent cells.
        """
        labels_pre_specifier =[
            "pre_synaptic_{}".format(c)
            for c in pre_cell_type_specifier]
        labels_post_specifier = [
            "post_synaptic_{}".format(c)
            for c in post_cell_type_specifier]
        cells = circuit_model.cells
        def _get_summary_connections(post_cell):
            return cells[
                list(pre_cell_type_specifier)
            ].rename(
                columns=lambda c: "pre_synaptic_{}".format(c)
            ).assign(
                number_pairs=np.in1d(
                    circuit_model.cells.index.values,
                    circuit_model.connectome.afferent_gids(post_cell.gid))
            ).groupby(
                labels_pre_specifier
            ).agg(
                ["size", "sum"]
            ).rename(
                columns={"size": "total", "sum": "connected"}
            ).assign(**{
                "post_synaptic_{}".format(p): post_cell[p] 
                for p in post_cell_type_specifier}
            ).reset_index(
            ).set_index(list(
                labels_pre_specifier + labels_post_specifier)
            )

        def _connection_probability(summary):
            """
            Compute connection probability between pairs
            """
            return summary.number_pairs.connected / summary.number_pairs.total

        return pd.concat(
            [_get_summary_connections(post_cell)
             for _, post_cell in cells.iterrows()]
        ).groupby(list(
            labels_pre_specifier + labels_post_specifier)
        ).agg(
            "sum"
        ).assign(
            connection_probability=_connection_probability )


In [ ]:
#sdirect = get_connection_probability(("mtype", ), ("mtype", ))
#sdirect.xs("L1_DAC", level="post_synaptic_mtype")

In [ ]:
l1dac_aff_mtypes.mtype.value_counts()